In [58]:
import pandas as pd
import numpy as np
import pickle

def transform(data, mean, std):
    return (data - mean) / std

def inverse_transform(data, mean, std):
    return (data * std) + mean

In [59]:
raw_data_path = "raw_dataset/PEMS04/PEMS04.npz"
adj_path = "raw_dataset/PEMS04/adj_PEMS04.pkl"
freq = 5 # minute

In [60]:
np.load(raw_data_path)['data'].shape

(16992, 307, 3)

In [61]:
raw_data = np.load(raw_data_path)['data'][..., 0]
raw_data = pd.DataFrame(raw_data, index=pd.date_range('2018-01-01',periods=16992, freq="5min"))
raw_data.head()

,0,1,2,3,4,5,6,7,8,9,...,297,298,299,300,301,302,303,304,305,306
2018-01-01 00:00:00,62.0,56.0,90.0,32.0,19.0,68.0,24.0,24.0,31.0,31.0,...,5.0,86.0,54.0,54.0,62.0,37.0,90.0,56.0,48.0,38.0
2018-01-01 00:05:00,61.0,40.0,94.0,25.0,16.0,64.0,22.0,7.0,29.0,33.0,...,0.0,86.0,46.0,41.0,59.0,38.0,91.0,51.0,30.0,35.0
2018-01-01 00:10:00,71.0,28.0,79.0,36.0,9.0,60.0,25.0,11.0,25.0,46.0,...,0.0,67.0,59.0,58.0,59.0,34.0,56.0,67.0,31.0,45.0
2018-01-01 00:15:00,86.0,47.0,94.0,46.0,11.0,58.0,35.0,17.0,28.0,46.0,...,22.0,101.0,82.0,75.0,74.0,75.0,42.0,126.0,64.0,81.0
2018-01-01 00:20:00,103.0,36.0,154.0,68.0,9.0,56.0,56.0,37.0,36.0,55.0,...,0.0,97.0,88.0,109.0,90.0,61.0,127.0,200.0,63.0,104.0


In [62]:
# variate
L, N = raw_data.shape
train_len = int(L * 0.6)
test_len = int(L * 0.2)
val_len = L - train_len - test_len

train_data = raw_data.values[0:train_len]
mean, std = train_data.mean(axis=0), train_data.std(axis=0)
norm_var = transform(raw_data.values, mean, std)

train_len, val_len, test_len

(10195, 3399, 3398)

In [63]:
raw_data.index[0].dayofweek

0

In [64]:
# time marker
tod_size = int((24 * 60) / freq) - 1
dow_size = 6
dom_size = 30
doy_size = 365

tod = np.array(list(map(lambda x: ((60 * x.hour + x.minute) / freq), raw_data.index))) # 0 ~ 287
dow = np.array(raw_data.index.dayofweek) # 0 ~ 6
dom = np.array(raw_data.index.day) - 1 # 0 ~ 30
doy = np.array(raw_data.index.dayofyear) - 1 # 0 ~ 181

time_marker = np.stack([tod, dow, dom, doy], axis=-1)
norm_time_marker = np.stack([tod / tod_size - 0.5, dow / dow_size - 0.5, dom / dom_size - 0.5, doy / doy_size - 0.5], axis=-1)

In [65]:
norm_var.shape

(16992, 307)

In [66]:
# adj_mat
with open(adj_path, 'rb') as f:
    adj = pickle.load(f, encoding="bytes")

In [67]:
np.savez("../../dataset/PEMS04/feature.npz", norm_var=norm_var, norm_time_marker=norm_time_marker, time_marker=time_marker)
np.savez("../../dataset/PEMS04/var_scaler_info.npz", mean=mean, std=std)
np.save("../../dataset/PEMS04/adj_mat.npy", adj[2])

In [68]:
train_len, val_len, test_len

(10195, 3399, 3398)

In [69]:
d = np.load("../../dataset/PEMS04/feature.npz")

In [70]:
d.files

['norm_var', 'norm_time_marker', 'time_marker']